In [1]:
import dataset
from tensorflow import estimator as tf_estimator
import models.losses as losses
import tensorflow as tf
from models.metrics import *
import models.cnn_autoencoder_model as cnnmodel
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
hparams = {
    # 数据路径
    'train_path': '../dataset/next_day_wildfire_spread_train*',
    'eval_path': '../dataset/next_day_wildfire_spread_eval*',
    'test_path': '../dataset/next_day_wildfire_spread_test*',
    
    # 特征
    'input_features': ['elevation', 'pdsi', 'NDVI', 'pr', 'sph', 'th', 'tmmn',
                  'tmmx', 'vs', 'erc', 'population', 'PrevFireMask'],
    'output_features': ['FireMask'],
    
    # 方位通道
    'azimuth_in_channel': None,
    'azimuth_out_channel': None,
    
    # 数据和模型参数
    'data_sample_size': 64,
    'sample_size': 32,
    'output_sample_size': 32,
    'batch_size': 128,
    'shuffle': False,
    'shuffle_buffer_size': 10000,
    'compression_type': None,
    'input_sequence_length': 1,
    'output_sequence_length': 1,
    'repeat': False,
    'clip_and_normalize': True,
    'clip_and_rescale': False,
    
    # 数据增强
    'random_flip': False,
    'random_rotate': False,
    'random_crop': False,
    'center_crop': True,
    
    # 其他参数
    'downsample_threshold': 0.0,
    'binarize_output': True
}

In [4]:
train_dataset = dataset.make_dataset(
    hparams,
    mode = tf_estimator.ModeKeys.TRAIN
)
val_dataset = dataset.make_dataset(
    hparams,
    mode = tf_estimator.ModeKeys.EVAL
)
test_dataset = dataset.make_dataset(
    hparams,
    mode = tf_estimator.ModeKeys.PREDICT
)

In [5]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=30,
    restore_best_weights=True,
)

## AutoEncoder

In [6]:
input_tensor = Input((32, 32, 12))
num_out_channels = 1
encoder_layers = [16,32]
decoder_layers = [32,16]
encoder_pools = [2,2]
decoder_pools = [2,2]
autoencoder_model = cnnmodel.create_model(
    input_tensor,
    num_out_channels,
    encoder_layers,
    decoder_layers,
    encoder_pools,
    decoder_pools,
)
autoencoder_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 12)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 16)   1728        ['input_1[0][0]']                
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 32, 32, 16)   0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_2 (Conv2D)              (None, 32, 32, 16)   192         ['input_1[0][0]']            

In [7]:
optimizer = tf.keras.optimizers.Adam()
autoencoder_model.compile(optimizer=optimizer,
              loss=losses.weighted_cross_entropy_with_logits_with_masked_class(pos_weight=3),
              metrics=[AUCWithMaskedClass(with_logits=True)])
history = autoencoder_model.fit(train_dataset, epochs=1000, validation_data=val_dataset, callbacks=[early_stopping])

Epoch 1/1000
118/118 [==============================] - 21s 35ms/step - loss: 0.2836 - auc_with_masked_class: 0.7639 - val_loss: 0.2968 - val_auc_with_masked_class: 0.8104
Epoch 2/1000
 35/118 [=======>......................] - ETA: 8s - loss: 0.2271 - auc_with_masked_class: 0.8596

KeyboardInterrupt: 

In [ ]:
autoencoder_model.evaluate(test_dataset)

14/14 [==============================] - 0s 25ms/step - loss: 0.2032 - auc_with_masked_class: 0.3595


[0.20317737758159637, 0.3595373034477234]

## U-Net

In [ ]:
def expend_as(tensor, rep):
     return Lambda(lambda x, repnum: K.repeat_elements(x, repnum, axis=3),
                          arguments={'repnum': rep})(tensor)

def double_conv_layer(x, filter_size, size, dropout, batch_norm=False):
    axis = 3
    conv = SeparableConv2D(size, (filter_size, filter_size), padding='same')(x)
    if batch_norm is True:
        conv = BatchNormalization(axis=axis)(conv)
    conv = Activation('relu')(conv)
    conv = SeparableConv2D(size, (filter_size, filter_size), padding='same')(conv)
    if batch_norm is True:
        conv = BatchNormalization(axis=axis)(conv)
    conv = Activation('relu')(conv)
    if dropout > 0:
        conv = Dropout(dropout)(conv)

    shortcut = Conv2D(size, kernel_size=(1, 1), padding='same')(x)
    if batch_norm is True:
        shortcut = BatchNormalization(axis=axis)(shortcut)

    res_path = add([shortcut, conv])
    return res_path

def encoder(inputs):
    num_filters = [16, 32, 64, 128]
    skip_connections = []
    x = inputs

    for i, f in enumerate(num_filters):
        a = double_conv_layer(x, 3, f, 0.1, True)
        skip_connections.append(a)
        x = MaxPooling2D(pool_size=(2, 2))(a)
    
    return x, skip_connections

def bottleneck(inputs):
    x = inputs
    f = 256
    
    x3 = double_conv_layer(x, 3, f, 0.1, True)
    
    return x3

def decoder(inputs, skip_connections):
    num_filters = [128, 64, 32, 16]
    skip_connections.reverse()
    x = inputs
    batch_norm = True
    
    for i, f in enumerate(num_filters):
        
        x_up = UpSampling2D(size=(2, 2), data_format="channels_last")(x)
        x_att = concatenate([x_up, skip_connections[i]], axis=-1)
        
        x = double_conv_layer(x_att, 3, f, 0.1, True)
    return x

def output(inputs):
    x = Conv2D(1, kernel_size=(1,1))(inputs)
    x = BatchNormalization()(x)
    # x = Activation('sigmoid')(x)
    
    return x

inputs = Input((32, 32, 12))
# s = layers.experimental.preprocessing.Rescaling(1.0 / 255)(inputs)
s = inputs
x, skip_1 = encoder(s)
x = bottleneck(x)
x = decoder(x, skip_1)
outputs = output(x)
unet_model = Model(inputs, outputs)
unet_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 12)  0           []                               
                                ]                                                                 
                                                                                                  
 separable_conv2d (SeparableCon  (None, 32, 32, 16)  316         ['input_2[0][0]']                
 v2D)                                                                                             
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 16)  64          ['separable_conv2d[0][0]']       
 alization)                                                                                 

In [ ]:
optimizer = tf.keras.optimizers.Adam()
unet_model.compile(optimizer=optimizer,
              loss=losses.weighted_cross_entropy_with_logits_with_masked_class(pos_weight=3),
              metrics=[AUCWithMaskedClass(with_logits=True)])
history = unet_model.fit(train_dataset, epochs=1000, validation_data=val_dataset, callbacks=[early_stopping])

Epoch 1/1000
118/118 [==============================] - 14s 59ms/step - loss: 0.6819 - auc_with_masked_class_1: 0.2517 - val_loss: 0.5865 - val_auc_with_masked_class_1: 0.0640
Epoch 2/1000
118/118 [==============================] - 11s 53ms/step - loss: 0.6078 - auc_with_masked_class_1: 0.3853 - val_loss: 0.5591 - val_auc_with_masked_class_1: 0.1325
Epoch 3/1000
118/118 [==============================] - 11s 54ms/step - loss: 0.5611 - auc_with_masked_class_1: 0.4154 - val_loss: 0.5305 - val_auc_with_masked_class_1: 0.2041
Epoch 4/1000
118/118 [==============================] - 11s 54ms/step - loss: 0.5200 - auc_with_masked_class_1: 0.4375 - val_loss: 0.5109 - val_auc_with_masked_class_1: 0.2637
Epoch 5/1000
118/118 [==============================] - 11s 52ms/step - loss: 0.4838 - auc_with_masked_class_1: 0.4532 - val_loss: 0.4818 - val_auc_with_masked_class_1: 0.3131
Epoch 6/1000
118/118 [==============================] - 11s 57ms/step - loss: 0.4507 - auc_with_masked_class_1: 0.4729 -

In [ ]:
unet_model.evaluate(test_dataset)

14/14 [==============================] - 0s 21ms/step - loss: 0.2220 - auc_with_masked_class_1: 0.3633


[0.2220226377248764, 0.36326098442077637]